In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.applications import VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19
from utils import to_grayscale_then_rgb
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam,SGD

In [ ]:
file_train_b_csv='..\Iris_train_basic_seg_list.csv'
df_train_basic= pd.read_csv(file_train_b_csv)
datagen=ImageDataGenerator(rescale=1./255.)#,preprocessing_function=to_grayscale_then_rgb)
df_train_basic['Label']= df_train_basic['Label'].astype(str)
df_train_basic


In [ ]:
file_val_csv='..\Iris_val_basic_seg_list.csv'
df_val= pd.read_csv(file_val_csv)
df_val['Label']= df_val['Label'].astype(str)


In [ ]:

train_generator = datagen.flow_from_dataframe(
    dataframe=df_train_basic,
    directory=r"D:\Users\jacob\AVS9\Data\saved_seg_class_images\train_basic",
    x_col="Name",
    y_col="Label",
    batch_size=10,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(224, 224))#540,810


In [ ]:
val_generator = datagen.flow_from_dataframe(
    dataframe=df_val,
    directory=r"D:\Users\jacob\AVS9\Data\saved_seg_class_images\val",
    x_col="Name",
    y_col="Label",
    batch_size=10,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(224, 224))


In [ ]:
file_test_csv='..\Iris_test_basic_seg_list.csv'
df_test= pd.read_csv(file_test_csv)
df_test['Label']= df_test['Label'].astype(str)
test_generator = datagen.flow_from_dataframe(
    dataframe=df_test,
    directory=r"D:\Users\jacob\AVS9\Data\saved_seg_class_images\test",
    x_col="Name",
    y_col="Label",
    batch_size=1,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(224, 224))

In [ ]:
# Definir la arquitectura de LeNet-5
input_shape=(224, 224, 1)

def build_LeNet5(input_shape, learning_ratea=0.001):
    input_layer = Input(shape=input_shape)

    x = Conv2D(6, kernel_size=(5, 5), activation='relu')(input_layer)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(16, kernel_size=(5, 5), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Flatten()(x)
    x = Dense(120, activation='relu')(x)
    x = Dense(84, activation='relu')(x)
    output_layer = Dense(2, activation='sigmoid')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=learning_ratea), loss='binary_crossentropy', metrics=['accuracy'])

    return model

model=build_LeNet5(input_shape,0.0001)

model.summary()


In [ ]:
learning_rate=0.001
input_shape=(224,224,3)
base_model = VGG19(weights='imagenet',input_shape=input_shape,include_top=True)
#base_model.get_layer(name='predictions')
# # Remove the original output layer (the 'predictions' layer)
output_layer = Dense(2, activation='sigmoid', name='custom_output')(base_model.layers[-2].output)
model = Model(inputs=base_model.input, outputs=(output_layer))
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
#model.reset_states()

In [ ]:
hist=model.fit(x=train_generator, steps_per_epoch=STEP_SIZE_TRAIN, validation_data=val_generator, 
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20,verbose=1)

In [ ]:
test_generator.reset()
pred=model.predict(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)
predicted_class_indices

In [ ]:
from sklearn.metrics import accuracy_score
labels=df_test['Label'].astype(int).tolist()
accuracy_score(labels,predicted_class_indices)